In [1]:
! pip install -q "transformers==4.44.0" "datasets==2.21.0" "evaluate==0.4.2" "accelerate==0.33.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
eli5 = load_dataset("eli5_category", split="train[:5000]", trust_remote_code=True)
eli5 = eli5.train_test_split(test_size=0.2)
eli5["train"][0]

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

{'q_id': '7g6bky',
 'title': 'How do Emulators like Dolphin make old GameCube games look almost like they are in 4K?',
 'selftext': "Basically what i'm asking is why does Super Mario Sunshine look blurry and terrible on it's native resolution but the water looks Crystal clear and amazing on the highest settings if the game was never intended to be seen that way?",
 'category': 'Technology',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dqgu3pe'],
  'text': ["The output of the gamecube was limited to the displays they knew they were going to use back then. I think the Gamecube's max output was 480p which is basically 640x480, because most TVs couldn't display any more detail than that. An emulator doesn't have those limitations. It has much more power to throw at the problem, and it has much higher output resolution, so it can take the original game code and render the output at greater resolutions, to truly from the ground up, increase the 3D detail. There's no way to impro

In [6]:
# Above, text field is actually nested inside answers. Extract the text subfield from its nested structure with the flatten method
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '7g6bky',
 'title': 'How do Emulators like Dolphin make old GameCube games look almost like they are in 4K?',
 'selftext': "Basically what i'm asking is why does Super Mario Sunshine look blurry and terrible on it's native resolution but the water looks Crystal clear and amazing on the highest settings if the game was never intended to be seen that way?",
 'category': 'Technology',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dqgu3pe'],
 'answers.text': ["The output of the gamecube was limited to the displays they knew they were going to use back then. I think the Gamecube's max output was 480p which is basically 640x480, because most TVs couldn't display any more detail than that. An emulator doesn't have those limitations. It has much more power to throw at the problem, and it has much higher output resolution, so it can take the original game code and render the output at greater resolutions, to truly from the ground up, increase the 3D detail. There's no way to im

In [7]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1128 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1031 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2153 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2167 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1282 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1145 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3377 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1195 > 1024). Running this sequence through the model will result in indexing errors


In [13]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

# Model

In [15]:
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Train

In [17]:
training_args = TrainingArguments(
    output_dir="clm",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,3.943100,3.835401
2,3.856900,3.826429
3,3.817300,3.823935


TrainOutput(global_step=3906, training_loss=3.8759494045118887, metrics={'train_runtime': 1039.0828, 'train_samples_per_second': 30.058, 'train_steps_per_second': 3.759, 'total_flos': 1020135176404992.0, 'train_loss': 3.8759494045118887, 'epoch': 3.0})

In [18]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 45.78


# Inference

In [19]:
prompt = "Somatic hypermutation allows the immune system to"

In [20]:
from transformers import pipeline

generator = pipeline("text-generation", model= model, tokenizer=tokenizer)
generator(prompt)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'generated_text': 'Somatic hypermutation allows the immune system to function normally and a number of treatments can vary. The only way it can prevent this is by making certain antibodies to other parts of the body known as malignant cells (like a spleen.) That'}]